## Image preparation

The below code can be used to transform the images in an input directory (Input_Dir) 
to the right size (e.g. 32x32 pixels) into an output directory (Output_Dir).

Note: Duplicates will be removed by evaluating the file hash

### Basic Parameter

Note: Do not rename any of the variables in this section, because the they are externally used in github action `Train Model`

In [1]:
# Parameters
Input_Dir = 'data_raw_all'
Output_Dir = 'data_resize_all'

# Target image size
target_size_x = 32
target_size_y = 32


### Load libraries and defaults

In [2]:
import glob
import os
from pathlib import Path

from PIL import Image


### Delete output directory

In [3]:
files = glob.glob(Output_Dir + '/*.jpg')
for f in files:
    os.remove(f)
print(str(len(files)) + " files have been deleted.")


1716 files have been deleted.


### Load files and resize

In [4]:
import hashlib

files = glob.glob(Input_Dir + '/*.jpg')
hashes={}
for i,aktfile in enumerate(files):
    if i%500==0:
        print(i, aktfile)
    test_image = Image.open(aktfile)
    hash=hashlib.sha256(test_image.tobytes()).hexdigest()
    if hash in hashes:
        hashes[hash].append(aktfile)
    else:
        hashes[hash]=[aktfile]
    test_image = test_image.resize((target_size_x, target_size_y), Image.LANCZOS)
    base=os.path.basename(aktfile)
    save_name = Output_Dir + '/' + base
    test_image.save(save_name, "JPEG", quality = 100)


0 data_raw_all\0.0_0.0.jpg
500 data_raw_all\2.0_0947_analog2_20200819-164047.jpg
1000 data_raw_all\4.3_2fad4056768f5d4dce52daa761b7e6db.jpg
1500 data_raw_all\6.6_3343_zeiger2_2019-11-19_01-27-03.jpg
2000 data_raw_all\8.3_cc4cc79801bbbac820b0b195cf066603.jpg


### Remove duplicate files

In [5]:
# duplicate files are a risk to the metrics, they pollute the validation dataset
for hash in hashes:
    if len(hashes[hash])>1:
        print(hashes[hash])    
        for duplicate in hashes[hash][1:]:
            # remove all except the first
            os.remove(duplicate)    
